## 本程式碼為取得戰車在現實生活中與物體的距離，以及在固定轉速與時間的情況下，戰車轉動的x軸pixel數，以便之後調整控制物體追蹤的參數

### 因為之後要透過程式碼控制戰車，我們先建立robot實體物件

In [3]:
from jetbot import Robot
import time

robot = Robot()

### 啟動相機並聯動widget讓我們能夠透過jupytor notebook看到實時串流影像

In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
import cv2


camera = Camera.instance(width=224, height=224)


image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)


display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

### 創建拍照存檔資料夾

In [2]:
import os

angle_dir = 'dataset/angleCalActualWorld'

try:
    os.makedirs(angle_dir)
except FileExistsError:
    print('Directories not created because they already exist')

Directories not created because they already exist


### 創建拍照按鈕以及即時取得當前拍照數量

In [3]:
button_layout = widgets.Layout(width='128px', height='64px')
capture_button = widgets.Button(description='add photo', button_style='success', layout=button_layout)
capture_count = widgets.IntText(layout=button_layout, value=len(os.listdir(angle_dir)))

### 定義拍照函數

In [4]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_photo():
    global angle_dir, angle_count
    save_snapshot(angle_dir)
    capture_count.value = len(os.listdir(angle_dir))
    
    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
capture_button.on_click(lambda x: save_photo())

### 顯示影像串流框及拍照按鈕

In [5]:
display(image)
display(widgets.HBox([capture_count, capture_button]))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

### 定義噴灑函數
### 透過固定噴頭測量噴灑範圍以及距離，並透過前面定義的拍照函數取得照片以便後續計算參數

In [ ]:
import Jetson.GPIO as GPIO
import time

output_pin = 21

def main():
    
# 設置為BCM模式

    GPIO.setmode(GPIO.BCM) 
    
# 將腳位設定為輸出，並預設為輸出高電位

    GPIO.setup(output_pin, GPIO.OUT, initial=GPIO.HIGH)

    print("Starting demo now! Press CTRL+C to exit")
    curr_value = GPIO.HIGH
    try:
        while True:
            
# 每隔4秒鐘切換

            time.sleep(4)
            print("Outputting {} to pin {}".format(curr_value, output_pin))
            GPIO.output(output_pin, curr_value)
            curr_value ^= GPIO.HIGH
    finally:
        GPIO.cleanup()

# if __name__ == '__main__':
#     main()

### 控制戰車轉動，並透過前面定義的拍照函數取得轉動後的照片以便後續計算參數

In [44]:
robot.forward(speed=0.3)
time.sleep(3)
robot.stop()

In [42]:
# forward_calibration
robot.set_motors(0.35, 0.35)
time.sleep(3)
robot.stop()

In [36]:
# 左轉(左輪向後, 右輪向前)
robot.left(speed=0.3)
time.sleep(1)
robot.stop()

In [29]:
# 右轉(右輪向後, 左輪向前)
robot.right(speed=0.3)
time.sleep(1)
robot.stop()

### 結束照片蒐集後記得要關閉戰車馬達以及相機

In [45]:
robot.stop()
camera.stop()

### 載入YOLOv5s模型

In [5]:
import torch
import torchvision

# 載入自行訓練的 YOLOv5 模型
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./models/yolov5s62best.pt', force_reload=True)

# 設定 IoU 門檻值
model.iou = 0.3

# 設定信心門檻值
model.conf = 0.5


def preprocess_yolo(camera_value):
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = cv2.resize(x, (224, 224))
    return x

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-9-27 Python-3.6.9 torch-1.8.0 CUDA:0 (NVIDIA Tegra X1, 3964MiB)



WARNING ⚠️ Python 3.7.0 is required by YOLOv5, but Python 3.6.9 is currently installed


Fusing layers... 
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


### 將前述拍攝的照片透過YOLOv5模型取得偵測物件Bounding Box 左上角、右下角 x, y座標

In [56]:
img_name = 'Spray_S'

img_1 = '/workspace/jetbot/notebooks/object_following/cockroach_imgs_internet/images_33.jpg'

results = model(img_1, size=224)

results.print()

results.save()

image_number = 0
object_number = 0

HighestConfidenceResult = results.pandas().xyxy[image_number].loc[object_number]
print(HighestConfidenceResult)

image 1/1: 149x339 (no detections)
Speed: 13.7ms pre-process, 82.0ms inference, 2.3ms NMS per image at shape (1, 3, 128, 224)
Saved 1 image to runs/detect/exp5


In [130]:
import cv2
import numpy as np

frame_width = 224
frame_height = 224

frame_central_x = int(frame_width/2)
frame_central_y = int(frame_height/2)

xmin = HighestConfidenceResult['xmin']
xmax = HighestConfidenceResult['xmax']
ymin = HighestConfidenceResult['ymin']
ymax = HighestConfidenceResult['ymax']

# 找出BBox中心點的x
w = xmax - xmin
central_x = int(xmin + (w/2))


# 找出BBox中心點的y
h = ymax - ymin
central_y = int(ymin + (h/2))


img = cv2.imread('/workspace/jetbot/notebooks/object_following/dataset/angleCalActualWorld/{}.jpg'.format(img_name))

cv2.line(img, (frame_central_x, 0), (frame_central_x, frame_height), (255, 0, 0), 2)  # Vertical line through frame central point
cv2.line(img, (0, frame_central_y), (frame_width, frame_central_y), (255, 0, 0), 2)  # Horizontal line through frame central point
cv2.circle(img, (central_x, central_y), 10, (255, 255, 0), 3)  # BBox central point
cv2.line(img, (frame_central_x, frame_central_y), (central_x, central_y), (0, 255, 0), 3)  # Line from central point to BBox central point
cv2.rectangle(img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 0, 255), 2)  # 

distance = np.sqrt((frame_central_x - central_x)**2 + (frame_central_y - central_y)**2)

print('h: {:.2f}'.format(h))
print('w: {:.2f}'.format(w))
print('central_x:', central_x)
print('central_y:', central_y)
print('distance: {:.2f}'.format(distance))


cv2.imwrite('{}_draw.jpg'.format(img_name), img)

h: 35.07
w: 33.19
central_x: 100
central_y: 119
distance: 13.89


True